<a href="https://colab.research.google.com/github/pidoxy/pipeops-hackathon/blob/main/PreProcessFinanceBotText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [13]:
!python -m spacy download en_core_web_sm

zsh:1: command not found: python


In [14]:
pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
import nltk
import string
from nltk.corpus import stopwords
import spacy
from spacy.lang.en import English

# Cleaning the text:

Using a programming language like Python and libraries like NLTK or spaCy to remove punctuation, special characters, unnecessary formatting, and stop words (common words like "the", "a", "and").

In [19]:

# Download stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
# Sample text from "Rich Dad, Poor Dad"
text = """
This book is about financial literacy. It's not about getting rich quick,
but about building a foundation for financial freedom.
"""

# 1. Remove punctuation
text = text.translate(str.maketrans('', '', string.punctuation))

# 2. Remove special characters
text = ''.join(char for char in text if char.isalnum() or char.isspace())

# 3. Tokenize (break into words)
tokens = nltk.word_tokenize(text)

# 4. Remove stop words
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

# 5. Join the filtered tokens back into a string
cleaned_text = ' '.join(filtered_tokens)

print(f"Original Text:\n{text}")
print(f"\nCleaned Text:\n{cleaned_text}")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/mac/nltk_data'
    - '/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/nltk_data'
    - '/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/share/nltk_data'
    - '/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# from google.colab import files

In [ ]:
# uploaded = files.upload()

 ## Read Book Text from Files:

In [ ]:
# Download resources if necessary
nltk.download('stopwords')
nltk.download('punkt')
spacy.load("en_core_web_sm")  # Load the small English model

# Read the book text from files
with open("rich_dad_poor_dad.txt", "r", encoding="utf-8") as f:
    rich_dad_text = f.read()

with open("why_c_students_work_for_a_students.txt", "r", encoding="utf-8") as f:
    c_students_text = f.read()

## Clean the Text (Using NLTK):

In [ ]:
def clean_text_nltk(text):
    """Cleans text using NLTK."""
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join(char for char in text if char.isalnum() or char.isspace())
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

# Clean the book texts
cleaned_rich_dad_text = clean_text_nltk(rich_dad_text)
cleaned_c_students_text = clean_text_nltk(c_students_text)

## Clean the Text (Using spaCy):

In [ ]:
# def clean_text_spacy(text):
#     """Cleans text using spaCy."""
#     nlp = spacy.load("en_core_web_sm")
#     doc = nlp(text)
#     tokens = [token.text for token in doc if token.is_alpha or token.is_space]
#     stop_words = nlp.Defaults.stop_words
#     filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
#     cleaned_text = ' '.join(filtered_tokens)
#     return cleaned_text

# # Clean the book texts
# cleaned_rich_dad_text = clean_text_spacy(rich_dad_text)
# cleaned_c_students_text = clean_text_spacy(c_students_text)

## Tokenization and Lemmatization (NLTK):

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:


import spacy
from spacy.lang.en import English

# ... (Previous code for reading and cleaning book text) ...

def tokenize_and_lemmatize_spacy(text):
    """Tokenizes and lemmatizes text using spaCy."""
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return lemmas

# Tokenize and lemmatize the book texts
lemmas_rich_dad = tokenize_and_lemmatize_spacy(cleaned_rich_dad_text)
lemmas_c_students = tokenize_and_lemmatize_spacy(cleaned_c_students_text)

In [ ]:

import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
import torch

In [ ]:
nltk.download('wordnet')

# very important to download


In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

## Creating Grammar

In [ ]:
# 1. Data Preparation
# - Assuming you have your cleaned and lemmatized text from the books in variables like `cleaned_rich_dad_text` and `cleaned_c_students_text`

def create_vocabulary(text):
    """Creates a vocabulary from the text."""
    tokens = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    vocabulary = set(lemmas)
    return vocabulary

# Create vocabularies for both books
rich_dad_vocabulary = create_vocabulary(cleaned_rich_dad_text)
c_students_vocabulary = create_vocabulary(cleaned_c_students_text)



In [ ]:
# 2. Fine-tuning Gemini
# - Choose a suitable Gemini model (e.g., "google/flan-t5-xl")
# - Load the model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# - Create a training dataset of question-answer pairs
#   - You can manually create these pairs from the books or use automatic question generation techniques
#   - Example: [(question1, answer1), (question2, answer2), ...]
training_data = [
    # Add your question-answer pairs here
    ("What is the key concept of Rich Dad, Poor Dad?", "The key concept is that financial literacy is crucial for building wealth and achieving financial freedom."),
    # ... more pairs
]



In [ ]:
!apt install accelerate

In [ ]:
def preprocess_data(data):
    """Preprocess training data."""
    processed_data = []
    for question, answer in data:
        input_ids = tokenizer(question, return_tensors="pt")["input_ids"].squeeze() # Add .squeeze() to remove unnecessary dimensions
        target_ids = tokenizer(answer, return_tensors="pt")["input_ids"].squeeze()
        processed_data.append({"input_ids": input_ids, "labels": target_ids})
    return processed_data

training_data = preprocess_data(training_data)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./fin_intel_model",
    num_train_epochs=3,
    per_device_train_batch_size=8, # reduced to 4 because of space
    learning_rate=2e-5,

)

# ... (rest of your training code)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_data, # No need for TensorDataset here
)

# print(target_ids.shape)
trainer.train()



In [ ]:
# - Save the fine-tuned model
trainer.save_model("./fin_intel_model")



In [ ]:
# 3. Building the Chatbot
# - Choose a chatbot platform
# - Example using Flask (for a web application)
from flask import Flask, request, jsonify
app = Flask(__name__)

In [ ]:


# - Load the fine-tuned Gemini model
model = AutoModelForSeq2SeqLM.from_pretrained("./fin_intel_model")


@app.route('/')
def index():
    return 'Index Page'

@app.route('/chat', methods=['POST'])
def chat():
    try:
      # Get the user's question from the request
      user_question = request.json['question']

      # Use the model to generate an answer
      inputs = tokenizer(user_question, return_tensors="pt")
      output = model.generate(**inputs)
      answer = tokenizer.decode(output[0], skip_special_tokens=True)

      return jsonify({'answer': answer})

    except KeyError:
        return jsonify({'error': 'Missing "question" in the request'}), 400
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5003)

# 4. Evaluation and Improvement
# - Test the chatbot with various questions
# - Gather feedback from users
# - Iteratively improve the chatbot based on feedback, new data, and fine-tuning

In [ ]:
test_data = [
    ("What is the difference between assets and liabilities in Rich Dad, Poor Dad?", "According to Rich Dad, Poor Dad, assets put money in your pocket, while liabilities take money out of your pocket. Assets are things like businesses, real estate, and stocks, while liabilities are things like cars, credit card debt, and mortgages."),
    ("What does Robert Kiyosaki teach about working for money?", "Robert Kiyosaki argues that working for money is a trap and that true wealth comes from owning assets that generate income."),
    ("How can I learn financial literacy?", "Financial literacy can be learned through books, courses, seminars, and online resources."),
    # ... Add more test questions ...
]

In [ ]:
# from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
# from flask import Flask, request, jsonify

# app = Flask(__name__)

# # Load your fine-tuned model
# model = AutoModelForSeq2SeqLM.from_pretrained("./fin_intel_model")
# tokenizer = AutoTokenizer.from_pretrained("./fin_intel_model")

# def evaluate_chatbot(test_data):
#     """Evaluates the chatbot on a test set."""
#     correct_answers = 0
#     total_questions = len(test_data)

#     for question, expected_answer in test_data:
#         # Get the chatbot's response
#         inputs = tokenizer(question, return_tensors="pt")
#         output = model.generate(**inputs)
#         answer = tokenizer.decode(output[0], skip_special_tokens=True)

#         # Compare the chatbot's answer to the expected answer
#         if answer.lower() == expected_answer.lower():  # Case-insensitive comparison
#             correct_answers += 1

#     accuracy = correct_answers / total_questions
#     print(f"Accuracy: {accuracy:.2f}")

# # Example usage
# evaluate_chatbot(test_data)